## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], 1)

<ipython-input-28-426713c1e390>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME"], 1)


In [ ]:
# Confirm columns were dropped
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
appl_counts = application_df["APPLICATION_TYPE"].value_counts()
appl_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choosing a cutoff value of 500, create a list of application types to be replaced by "Other"
application_types_to_replace = appl_counts[appl_counts<500].index
application_types_to_replace

Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [ ]:
# Replace those application types in dataframe with "Other"
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Confirm binning was successful - Other replaced application types with fewer than 500 entries
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# Zoom in on smallest CLASSIFICATION value counts
class_counts[class_counts<100]

C2800    95
C7100    75
C1300    58
C1280    50
C1230    36
C1400    34
C7200    32
C2300    32
C1240    30
C8000    20
C7120    18
C1500    16
C1800    15
C6000    15
C1250    14
C8200    11
C1238    10
C1278    10
C1235     9
C1237     9
C7210     7
C2400     6
C1720     6
C4100     6
C1257     5
C1600     5
C1260     3
C2710     3
C0        3
C3200     2
C1234     2
C1246     2
C1267     2
C1256     2
C2190     1
C4200     1
C2600     1
C5200     1
C1370     1
C1248     1
C6100     1
C1820     1
C1900     1
C1236     1
C3700     1
C2570     1
C1580     1
C1245     1
C2500     1
C1570     1
C1283     1
C2380     1
C1732     1
C1728     1
C2170     1
C4120     1
C8210     1
C2561     1
C4500     1
C2150     1
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = class_counts[class_counts<100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Confirm binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Review the types of data in each column
appl_dtypes = application_df.dtypes
appl_dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [ ]:
# Create list of columns with object data types
categ_cols = appl_dtypes[appl_dtypes == "object"].index.to_list()
categ_cols

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
new_appl_df = pd.get_dummies(application_df,columns=categ_cols)
new_appl_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features (X) and target (y) arrays
X = new_appl_df.drop(columns="IS_SUCCESSFUL").values
y = new_appl_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# View X_train
X_train

array([[   1, 5000,    0, ...,    0,    1,    0],
       [   1, 5000,    0, ...,    0,    1,    0],
       [   1, 5000,    0, ...,    0,    1,    0],
       ...,
       [   1, 5000,    0, ...,    0,    1,    0],
       [   1, 5000,    0, ...,    0,    1,    0],
       [   1, 5000,    0, ...,    0,    1,    0]])

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim=49))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                4000      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,461
Trainable params: 6,461
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
## Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6037 - accuracy: 0.7020
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5690 - accuracy: 0.7241
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5630 - accuracy: 0.7270
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5683 - accuracy: 0.7214
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5612 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5883 - accuracy: 0.7059
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5589 - accuracy: 0.7288
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7282
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7310
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5562 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5703 - accuracy: 0.7324 - 420ms/epoch - 2ms/step
Loss: 0.5703117847442627, Accuracy: 0.7323614954948425


In [ ]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity.h5')